In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.2/375.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/

In [2]:
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 133.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.24.0
    Uninstalling trl-0.24.0:
      Successfully uninstalled trl-0.24.0


In [3]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
from huggingface_hub import login
login('')

In [5]:
base_model_name = "unsloth/Meta-Llama-3.1-8B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name,
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

model.load_adapter(
    "robiophantom/Llama-3.1-8B-lora-NvidiaLab", # adapter path
    adapter_name="lora"
)
model.set_adapter("lora")


==((====))==  Unsloth 2025.12.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [6]:
from transformers import TextStreamer

In [10]:
# 2. Inference prompt (NO answer slot)
# -----------------------------
inference_prompt = """Below is an instruction that describes a task.

### Instruction:
You are AgroShakti, an AI-powered agricultural assistant created to help Indian farmers.
You are an expert in agriculture, farming schemes, crop management, soil health, irrigation,
pest and disease control, weather-based advisories, and government programs such as KCC.

Follow these rules strictly:
1. Provide accurate, practical, and farmer-friendly advice.
2. Use simple language that is easy for farmers to understand.
3. Explain concepts step by step.
4. Give examples, best practices, and precautions.
5. Respond in detail (minimum 500 words).
6. Do NOT provide medical, legal, or veterinary advice.
7. Do NOT give unsafe chemical dosages.
8. If unsure, clearly say so.
9. Do not give any contact number.

Always respond as AgroShakti with a supportive tone.

### Farmer's Question:
{question}

### AgroShakti's Answer:
"""

# -----------------------------
# 3. Prepare input
# -----------------------------
question = "what is pm kisan yoajana and how can i register in it?"

inputs = tokenizer(
    inference_prompt.format(question=question),
    return_tensors="pt",
    truncation=True
).to("cuda")

# -----------------------------
# 4. Stream output
# -----------------------------
streamer = TextStreamer(tokenizer, skip_prompt=True)

# -----------------------------
# 5. Generate (LONG OUTPUT)
# -----------------------------
with torch.no_grad():
    _ = model.generate(
        **inputs,
        streamer=streamer,

        max_new_tokens=512,
        min_new_tokens=128,

        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        repetition_penalty=1.1,

        early_stopping=False,
    )

The Pradhan Mantri Kisan Samman Nidhi Yojana is a scheme launched by the Government of India to provide financial assistance to small and marginal farmers. The benefit amount under this scheme is Rs.6000 per annum. To get the benefit of this scheme, you need to have Aadhaar card, bank account, land record, mobile number and you should be a farmer. For more information, you can contact the office of the Deputy Director of Agriculture at your district or call the helpline number 011- / 1800-115-526. You can also visit the official website of PM Kisan Yojana -<|end_of_text|>
